In [4]:
import os

In [5]:
%pwd

'/home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'/home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [9]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Reading yaml file from: /home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/config/config.yaml
[2023-07-06 23:04:53,208: INFO: common: yaml file: /home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/config/config.yaml loaded successfully]
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Reading yaml file from: /home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/params.yaml
[2023-07-06 23:04:53,210: INFO: common: yaml file: /home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/params.yaml loaded successfully]
[2023-07-06 23:04:53,212: INFO: common: created directory at: artifacts]
[2023-07-06 23:04:53,214: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoin

2023-07-06 23:04:53.218209: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-07-06 23:04:53.218256: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2023-07-06 23:04:53.219136: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [15]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-07-06-23-05-14'